# TP 3 - Partie 1 : Fondamentaux PyTorch & HuggingFace 🤗

Dans ce notebook, nous allons explorer les briques de base du deep learning avec PyTorch et découvrir l'écosystème HuggingFace.

**Objectifs :**
1. Comprendre ce qu'est un tenseur et comment représenter des données (images, texte)
2. Comprendre ce qu'est une couche Linear et comment s'empilent les couches
3. Explorer l'architecture d'un vrai modèle de NLP
4. Fine-tuner un modèle léger sur une tâche de classification

⚠️ **Contrainte matérielle** : Nous utilisons des modèles légers adaptés aux PCs de faculté.

## 1. Imports et setup

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests

# HuggingFace
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from datasets import load_dataset

# Vérifier le device disponible
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Device utilisé : {device}")
print(f"Version PyTorch : {torch.__version__}")

## 2. Les Tenseurs : la brique de base du Deep Learning

Un **tenseur** est une structure de données multi-dimensionnelle. C'est la représentation universelle en deep learning.

**Analogie avec les cours précédents :**
- **Image** = Tenseur 3D (Hauteur × Largeur × Canaux) ou (Canaux × Hauteur × Largeur)
- **Texte tokenisé** = Tenseur 1D (liste d'indices de tokens)
- **Batch d'images** = Tenseur 4D (Batch × Canaux × H × W)

### 2.1 Création et manipulation de tenseurs

In [ ]:
# Créer un tenseur simple
scalar = torch.tensor(42)
vector = torch.tensor([1, 2, 3, 4, 5])
matrix = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

print(f"Scalaire : {scalar}, shape : {scalar.shape}")
print(f"Vecteur : {vector}, shape : {vector.shape}")
print(f"Matrice :\n{matrix}, shape : {matrix.shape}")

# Tenseurs aléatoires (comme les poids initialisés d'un réseau)
random_tensor = torch.randn(3, 4)  # 3 lignes, 4 colonnes, distribution normale
print(f"\nTenseur aléatoire (3×4) :\n{random_tensor}")

### 2.2 Une image est un tenseur d'entiers !

Téléchargeons une image simple et regardons sa représentation en tant que tenseur.

In [ ]:
# Télécharger une image exemple
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/300px-PNG_transparency_demonstration_1.png"
img = Image.open(requests.get(url, stream=True).raw)

# Convertir en tenseur PyTorch
# Une image RGB a 3 canaux : Rouge, Vert, Bleu
img_array = np.array(img)
img_tensor = torch.from_numpy(img_array)

print(f"Shape de l'image (H×W×C) : {img_tensor.shape}")
print(f"Type de données : {img_tensor.dtype}")
print(f"Valeurs min/max : {img_tensor.min()} / {img_tensor.max()}")

# Affichons quelques pixels
print(f"\nPixels du coin supérieur gauche (10×10) :")
print(img_tensor[:10, :10, 0])  # Canal rouge uniquement

# Visualisation
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_tensor.numpy())
plt.title("Image originale")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_tensor[:, :, 0].numpy(), cmap='Reds')
plt.title("Canal Rouge uniquement")
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n👉 Chaque pixel est juste un nombre entier entre 0 et 255 !")

## 3. La couche Linear (Fully Connected / Dense)

**Rappel du cours :** Une couche Linear (ou fully connected) est la transformation la plus simple d'un réseau de neurones.

**Formule :** `y = x @ W^T + b`

- **x** : entrée de dimension `(batch_size, in_features)`
- **W** : matrice de poids de dimension `(out_features, in_features)`
- **b** : vecteur de biais de dimension `(out_features,)`
- **y** : sortie de dimension `(batch_size, out_features)`

### 3.1 Créons une couche Linear from scratch

In [ ]:
# Paramètres de notre couche
in_features = 4   # Dimension d'entrée
out_features = 3  # Dimension de sortie

# Création d'une couche Linear avec PyTorch
linear_layer = nn.Linear(in_features, out_features)

print("=== Architecture de la couche ===")
print(f"Entrée : {in_features} dimensions")
print(f"Sortie : {out_features} dimensions")
print(f"\nNombre total de paramètres : {sum(p.numel() for p in linear_layer.parameters())}")

print("\n=== Poids (W) ===")
print(f"Shape : {linear_layer.weight.shape}")  # (out_features, in_features)
print(f"Valeurs :\n{linear_layer.weight}")

print("\n=== Biais (b) ===")
print(f"Shape : {linear_layer.bias.shape}")  # (out_features,)
print(f"Valeurs : {linear_layer.bias}")

In [ ]:
# Passons une entrée à travers la couche
x = torch.tensor([[1.0, 2.0, 3.0, 4.0]])  # 1 échantillon, 4 features
print(f"Entrée x : {x}, shape : {x.shape}")

# Forward pass
y = linear_layer(x)
print(f"\nSortie y : {y}, shape : {y.shape}")

# Vérifions manuellement le calcul
# y = x @ W^T + b
manual_y = x @ linear_layer.weight.T + linear_layer.bias
print(f"\nCalcul manuel : {manual_y}")
print(f"Résultats identiques ? {torch.allclose(y, manual_y)}")

# Visualisation de la transformation
print(f"\n📊 Transformation : {in_features}D → {out_features}D")
print(f"   Chaque sortie est une combinaison linéaire de toutes les entrées")

### 3.2 Empiler des couches : création d'un MLP simple

Un réseau de neurones = empilement de couches avec des fonctions d'activation entre elles.

In [ ]:
# Définition d'un MLP (Multi-Layer Perceptron) simple
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.activation = nn.ReLU()  # Fonction d'activation non-linéaire
        self.layer2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        return x

# Instancier le modèle
model = SimpleMLP(input_dim=10, hidden_dim=20, output_dim=2)

print("=== Architecture du MLP ===")
print(model)

# Compter les paramètres par couche
print("\n=== Paramètres par couche ===")
total = 0
for name, param in model.named_parameters():
    print(f"{name:20s} : {list(param.shape)} → {param.numel()} paramètres")
    total += param.numel()
print(f"\nTotal : {total} paramètres")

## 4. Exploration d'un vrai modèle HuggingFace

Maintenant, chargeons un modèle de NLP léger et explorons son architecture couche par couche.

**Modèle choisi :** `distilbert-base-uncased-finetuned-sst-2-english` (DistilBERT small, ~66M params)

C'est une version allégée de BERT, parfaite pour les PCs de faculté.

In [ ]:
# Charger un modèle léger pré-entraîné
# DistilBERT = version allégée de BERT (40% moins de params, 60% plus rapide)

model_name = "distilbert-base-uncased"  # Modèle de base sans fine-tuning

print("Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Chargement du modèle...")
bert_model = AutoModel.from_pretrained(model_name)

# Compter les paramètres totaux
total_params = sum(p.numel() for p in bert_model.parameters())
trainable_params = sum(p.numel() for p in bert_model.parameters() if p.requires_grad)

print(f"\n✅ Modèle chargé : {model_name}")
print(f"   Paramètres totaux : {total_params:,} (~{total_params/1e6:.1f}M)")
print(f"   Paramètres entraînables : {trainable_params:,}")

In [ ]:
# Explorer l'architecture du modèle
print("=== Architecture complète ===")
print(bert_model)

print("\n=== Structure hiérarchique ===")
for name, module in bert_model.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"\n📁 {name}")
    print(f"   Type : {module.__class__.__name__}")
    print(f"   Paramètres : {params:,}")
    
    # Afficher les sous-modules (premier niveau)
    for sub_name, sub_module in list(module.named_children())[:3]:  # Limiter à 3
        sub_params = sum(p.numel() for p in sub_module.parameters())
        print(f"   └── {sub_name} : {sub_module.__class__.__name__} ({sub_params:,} params)")
    
    if len(list(module.named_children())) > 3:
        print(f"   └── ... et {len(list(module.named_children())) - 3} autres")

In [ ]:
# Explorer en détail les poids d'une couche d'attention
print("=== Deep Dive : Première couche d'attention ===")

# Accéder à la première couche du transformer
first_layer = bert_model.transformer.layer[0]

print(f"Structure de la couche 0 :")
print(first_layer)

# Explorer les poids de l'attention
attention = first_layer.attention
print("\n=== Poids de l'attention ===")

for name, param in attention.named_parameters():
    print(f"\n🔧 {name}")
    print(f"   Shape : {param.shape}")
    print(f"   Valeurs (premiers éléments) : {param.flatten()[:5].tolist()}")
    print(f"   Statistiques : mean={param.mean():.4f}, std={param.std():.4f}")

In [ ]:
# Passer une phrase à travers le modèle et observer les shapes
text = "Deep learning is fascinating!"

# Tokenization
inputs = tokenizer(text, return_tensors="pt")
print(f"Texte : '{text}'")
print(f"\nTokens : {inputs['input_ids'][0].tolist()}")
print(f"Token strings : {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")

# Passage dans le modèle
with torch.no_grad():  # Pas besoin de gradients pour l'inférence
    outputs = bert_model(**inputs)

print(f"\n=== Shapes des sorties ===")
print(f"Dernière couche cachée : {outputs.last_hidden_state.shape}")
print(f"   → [batch_size={outputs.last_hidden_state.shape[0]}, ")
print(f"      seq_len={outputs.last_hidden_state.shape[1]}, ")
print(f"      hidden_dim={outputs.last_hidden_state.shape[2]}]")

print(f"\nExplication :")
print(f"   - Batch size : 1 (une seule phrase)")
print(f"   - Seq len : {outputs.last_hidden_state.shape[1]} tokens")
print(f"   - Hidden dim : 768 (dimension des embeddings de DistilBERT)")

## 5. Fine-tuning sur une tâche de classification

Nous allons fine-tuner DistilBERT sur le dataset SST-2 (sentiment analysis).

**Important :** Nous utilisons un sous-ensemble du dataset pour que l'entraînement soit rapide sur CPU.

In [ ]:
# Charger le dataset SST-2 (petit, rapide à télécharger)
print("Chargement du dataset SST-2...")
dataset = load_dataset("glue", "sst2")

print(f"\nStructure du dataset :")
print(dataset)

# Réduire la taille pour l'entraînement rapide
small_train = dataset["train"].shuffle(seed=42).select(range(500))  # 500 exemples
small_val = dataset["validation"].shuffle(seed=42).select(range(100))  # 100 exemples

print(f"\nSous-ensemble pour l'entraînement :")
print(f"   Train : {len(small_train)} exemples")
print(f"   Validation : {len(small_val)} exemples")

# Exemple
print(f"\nExemple d'entrée :")
print(f"   Texte : {small_train[0]['sentence']}")
print(f"   Label : {small_train[0]['label']} ({'positif' if small_train[0]['label'] == 1 else 'négatif'})")

In [ ]:
# Tokeniser le dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=128)

tokenized_train = small_train.map(tokenize_function, batched=True)
tokenized_val = small_val.map(tokenize_function, batched=True)

# Charger le modèle pour la classification
model_clf = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2
)

print(f"Modèle de classification chargé")
print(f"   Classes : négatif (0), positif (1)")
print(f"   Classifier head : {model_clf.classifier}")

In [ ]:
# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # 2 epochs pour aller vite
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="no",  # Ne pas sauvegarder pour gagner du temps
    report_to="none",  # Pas de wandb/tensorboard
)

# Data collator pour le padding dynamique
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Créer le Trainer
trainer = Trainer(
    model=model_clf,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

print("Démarrage de l'entraînement...")
print("(Cela peut prendre 2-5 minutes sur CPU)")
trainer.train()

In [ ]:
# Tester sur quelques phrases
test_sentences = [
    "This movie is absolutely fantastic!",
    "I hate this film, it's terrible.",
    "The acting was okay, nothing special."
]

print("=== Prédictions ===")
model_clf.eval()
for sentence in test_sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model_clf(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred].item()
    
    sentiment = "positif" if pred == 1 else "négatif"
    print(f"\n'{sentence}'")
    print(f"   → Sentiment : {sentiment} (confiance : {confidence:.2%})")

## 🎯 Récapitulatif

Dans ce notebook, nous avons vu :

1. **Les tenseurs** : Images = tenseurs d'entiers (H×W×C), Texte = tenseurs d'indices
2. **Les couches Linear** : Transformation linéaire `y = x @ W^T + b`
3. **L'architecture** : Empilement de couches (Linear → Activation → Linear...)
4. **Exploration de modèle** : Inspection des poids et des shapes dans DistilBERT
5. **Fine-tuning** : Adaptation d'un modèle pré-entraîné à une nouvelle tâche

**Prochaine étape** : Partie 2 - Transfer Learning en Vision avec des modèles légers !

## ✏️ Exercices optionnels

1. **Modifier l'architecture** : Ajouter une couche cachée supplémentaire au SimpleMLP
2. **Inspecter les gradients** : Après un backward(), afficher `linear_layer.weight.grad`
3. **Essayer d'autres modèles** : Charger `prajjwal1/bert-tiny` (encore plus petit) et comparer
4. **Batch processing** : Passer plusieurs phrases en même temps et observer le batch size